In [1]:
from pathlib import Path
import os
import sys

sys.path.append(Path(os.path.abspath('')).parent.parent)
os.chdir(Path(os.path.abspath('')).parent.parent)

import numpy as np
import pandas as pd 
import xarray as xr
import geopandas as gpd 
from shapely import box

import config.settings as cset

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colormaps as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from config.settings import (labelsize_small, 
                            labelsize_medium, 
                            labelsize_large,
                            )

from tqdm.notebook import tqdm

### Processing GDP 

In [2]:
gdp_dfs = []

files = cset.GDP_files
dataset_names = cset.GDP_interim_dataset_names
cset.path_INTERIM_GDP.mkdir(parents = True, exist_ok = True)

for i, file in tqdm(enumerate(files), total  = 3):
    df = pd.read_csv(file).loc[:, ['iso', 'year', 'run', 'scen', 'SSP2_clim', 'SSP2']]
    df = df.loc[:, ['iso', 'year', 'run', 'scen', 'SSP2_clim', 'SSP2']].copy()
    # if not i == 2:
    #     df['run'] = df['run'].str.split('_').str[1]
    df.loc[df.scen == 'Refp5', 'scen'] = 'Ref1p5'
    df['run'] = [f.split('_')[-1] for f in df['run'].astype(str)]
    df = df[df.scen != 'Ref1p5']
    df.loc[df.scen == 'Refv2', 'scen'] = 'Ref'
    df['scen_run'] = df['scen'] + '_' + df['run'].astype(str)
    df['SSP2_diff'] = df['SSP2_clim']/df['SSP2']
    df.reset_index(drop = True, inplace = True)
    df.to_csv(cset.path_INTERIM_GDP / f'{dataset_names[i]}', index = False)
    gdp_dfs.append(df)
    del df 

  0%|          | 0/3 [00:00<?, ?it/s]

### Computing Peak GDP impacts 

In [8]:
gmt_ds = pd.read_csv(cset.path_MESMER_char / f'gmt_dataset.csv', index_col = 0)
peak_gmt_year = gmt_ds.idxmax(axis = 0).astype(int)

In [3]:
country_isos = gdp_dfs[0]['iso'].unique()
gmt_ds = pd.read_csv(cset.path_MESMER_char / f'gmt_dataset.csv', index_col = 0)
peak_gmt_year = gmt_ds.idxmax(axis = 0).astype(int)
peak_gdp_impacts = [pd.DataFrame(columns = gmt_ds.columns, index = country_isos) for _ in range(3)]
for i, gdp_df in enumerate(gdp_dfs):
    for year in tqdm(peak_gmt_year.unique(), total = len(peak_gmt_year.unique())):
        scenarios = peak_gmt_year[peak_gmt_year == year].index
        filtered_df = gdp_df[(gdp_df['year'] == year) & (gdp_df['scen_run'].isin(scenarios))]
        for scenario in scenarios: 
            peak_gdp_impacts[i][scenario] = filtered_df[(filtered_df['scen_run'] == scenario)].set_index('iso')['SSP2_diff'].values
    cset.path_OUT_PW.mkdir(parents = True, exist_ok = True)
    peak_gdp_impacts[i].to_csv(cset.path_OUT_PW / cset.PW_dataset_names[i])

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

In [5]:
thrshld = 1.35
thrshld_str = '135'
# loading & preparing glmt data 
gmt_char_df = pd.read_csv(cset.path_MESMER_char / f'gmt_characteristics_thsld_{thrshld_str}.csv', index_col = 0)

# Function to extract per-ensemble-member country median for each scenario group
def get_ensemble_medians(df, dataset_label):
    rows = []
    for scenario_prefix in cset.focus_scenarios:  # Add more as needed
        scenario_cols = [col for col in df.columns if col.startswith(scenario_prefix)]
        for col in scenario_cols:
            country_median = df[col].median()
            rows.append({
                'Scenario': scenario_prefix,
                'EnsembleMember': col,
                'MedianValue': country_median,
                'Dataset': dataset_label
            })
    return pd.DataFrame(rows)

# Combine medians for all datasets
ensemble_medians_df = pd.concat([
    get_ensemble_medians(df, f'{dataset_names[i]}')
    for i, df in enumerate(peak_gdp_impacts)
])

# plotting mean warming between 2015-2030 against median GDP impacts dring peak warming year 
ensemble_medians_df['gmt_ntwr'] = gmt_char_df.loc[ensemble_medians_df.EnsembleMember.values, 'gmt_ntwr'].values